In [1]:
import pandas as pd

/home/developer/python-dev/airflow_project1/env/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
from sqlalchemy import create_engine

In [4]:
# # setup default arguments
# default_args = {
#     'download_url': 'https://data.cityofnewyork.us/resource/9w7m-hzhe.json?inspection_date=',
#     'download_date_days_ago': -3,
#     'download_date_months_ago': -3,
#     'db_hostname': '192.168.1.221',
#     'db_username': 'restuser',
#     'db_password': 'restuser',
#     'db_database': 'restaurantdb',
# }


# setup default arguments
default_args = {
    'download_url': 'https://data.cityofnewyork.us/resource/9w7m-hzhe.json?inspection_date=',
    'download_date_days_ago': -3,
    'download_date_months_ago': -4,
    'db_hostname': 'localhost',
    'db_username': 'root',
    'db_password': 'workadmin',
    'db_database': 'restaurant_inbox',
}



In [5]:
engine = create_engine("mysql+pymysql://" + default_args['db_username'] + ":" + default_args['db_password'] + "@" + default_args['db_hostname'] + "/" + default_args['db_database'] + "", echo=False)
conn = engine.connect()

In [7]:
# db = dbUtil(default_args['db_hostname'], default_args['db_username'], default_args['db_password'], default_args['db_database'], True)
sql_string = """
select distinct 
v_business_number as business_number,
v_business_title as business_title,
v_address_number as address_number,
v_address_name as address_name,
v_zipcode as zipcode,
max(v_record_date) as record_date_max
from restaurant_inbox.restaurant_inbox a 
where v_business_number is not null
group by
v_business_number,
v_business_title,
v_address_number,
v_address_name,
v_zipcode
"""

sql_string = """
select distinct 
v_business_number as business_number,
v_business_title as business_title,
v_address_number as address_number,
v_address_name as address_name,
v_zipcode as zipcode,
v_boro as boro,
v_record_date as record_date
from restaurant_inbox a
order by v_business_number, v_record_date
"""


In [8]:
df = pd.read_sql(sql_string, conn)

In [10]:
df_boro = df[['boro']]
df_boro.columns = ['boro_name']
df_boro_nodup = df_boro.drop_duplicates()

In [11]:
df_boro_nodup.head(10)

,boro_name
0,None
1,BRONX
4,BROOKLYN
9,MANHATTAN
14,QUEENS
78,STATEN ISLAND
60816,Missing


In [193]:
df_boro_nodup.to_sql('dim_boro', con=engine, if_exists='append', index=False)

In [13]:
df.groupby(['business_number'])['record_date'].transform(max).head(10)

0                   NaT
1   2018-05-16 06:01:08
2   2018-05-16 06:01:08
3   2018-05-16 06:01:08
4   2018-03-23 06:01:07
5   2018-03-23 06:01:07
6   2018-03-23 06:01:07
7   2018-03-23 06:01:07
8   2018-03-23 06:01:07
9   2018-05-30 06:01:03
Name: record_date, dtype: datetime64[ns]

In [14]:
#df.groupby(['business_number'])['record_date'].idxmax()
df_business_latest = df[df['record_date'] == df.groupby(['business_number'])['record_date'].transform(max)]

In [35]:
df_business_latest.head(30)

,business_number,business_title,address_number,address_name,zipcode,boro,record_date
3,30075445.0,MORRIS PARK BAKE SHOP,1007,MORRIS PARK AVE,10462,BRONX,2018-05-16 06:01:08
8,30112340.0,WENDY'S,469,FLATBUSH AVENUE,11225,BROOKLYN,2018-03-23 06:01:07
11,30191841.0,DJ REYNOLDS PUB AND RESTAURANT,351,WEST 57 STREET,10019,MANHATTAN,2018-05-30 06:01:03
13,40356018.0,RIVIERA CATERERS,2780,STILLWELL AVENUE,11224,BROOKLYN,2018-01-03 06:00:54
15,40356151.0,BRUNOS ON THE BOULEVARD,8825,ASTORIA BOULEVARD,11369,QUEENS,2018-01-03 06:00:54
18,40356483.0,WILKEN'S FINE FOOD,7114,AVENUE U,11234,BROOKLYN,2018-05-16 06:01:08
21,40356731.0,TASTE THE TROPICS ICE CREAM,1839,NOSTRAND AVENUE,11226,BROOKLYN,2018-05-30 06:01:03
24,40357217.0,WILD ASIA,2300,SOUTHERN BOULEVARD,10460,BRONX,2018-06-04 06:00:46
26,40359480.0,1 EAST 66TH STREET KITCHEN,1,EAST 66 STREET,10065,MANHATTAN,2018-01-03 06:00:54
28,40359705.0,NATHAN'S FAMOUS,1310,SURF AVENUE,11224,BROOKLYN,2018-01-03 06:00:54


In [36]:
s = df_business_latest['business_number']
[s.value_counts() > 1] == True

False

In [37]:
df_business_latest.head()

,business_number,business_title,address_number,address_name,zipcode,boro,record_date
3,30075445.0,MORRIS PARK BAKE SHOP,1007,MORRIS PARK AVE,10462,BRONX,2018-05-16 06:01:08
8,30112340.0,WENDY'S,469,FLATBUSH AVENUE,11225,BROOKLYN,2018-03-23 06:01:07
11,30191841.0,DJ REYNOLDS PUB AND RESTAURANT,351,WEST 57 STREET,10019,MANHATTAN,2018-05-30 06:01:03
13,40356018.0,RIVIERA CATERERS,2780,STILLWELL AVENUE,11224,BROOKLYN,2018-01-03 06:00:54
15,40356151.0,BRUNOS ON THE BOULEVARD,8825,ASTORIA BOULEVARD,11369,QUEENS,2018-01-03 06:00:54


In [38]:
truncate_sql = """
truncate table trans_dim_business;
"""
update_res = pd.io.sql.execute(truncate_sql, conn)


In [39]:
df_business_latest.to_sql('trans_dim_business', con=engine, if_exists='append', index=False)

In [40]:
update_sql = """
update trans_dim_business a join dim_boro b on a.boro = b.boro_name 
set a.boro_id = b.boro_id
"""

In [41]:
update_res = pd.io.sql.execute(update_sql, conn)

In [42]:
truncate_sql = """
truncate table dim_business
"""

In [43]:
pd.io.sql.execute(truncate_sql, conn)

In [44]:
insert_sql = """
insert into dim_business (business_number, business_title, address_number, address_name, zipcode, boro_id, record_date, create_date, modify_date)
select business_number, business_title, address_number, address_name, zipcode, boro_id, record_date, now(), now() from trans_dim_business
"""

In [45]:
insert_sql

'\ninsert into dim_business (business_number, business_title, address_number, address_name, zipcode, boro_id, record_date, create_date, modify_date)\nselect business_number, business_title, address_number, address_name, zipcode, boro_id, record_date, now(), now() from trans_dim_business\n'

In [46]:
pd.io.sql.execute(insert_sql, conn)